<a href="https://colab.research.google.com/github/shaking54/CS114.K21.KHTN/blob/master/Folio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
with zipfile.ZipFile("/content/drive/My Drive/Folio Leaf Dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")

In [2]:
import tensorflow as tf
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
tf.__version__

'2.2.0'

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
def load_images_from_folder(pathdir):
    images = []
    labels = []
    for folder in os.listdir(pathdir):
      for filename in os.listdir(pathdir+'/'+folder):
          if filename.endswith('.jpg') or filename.endswith('.JPG') :
            img = image.load_img(pathdir+'/'+folder+'/'+filename,grayscale=True, target_size=(256,128,3))
            #img = image.img_to_array(img)
            #img=img/255
            images.append(img)
            labels.append(folder)
    return images, labels


Using TensorFlow backend.


In [4]:
X_train, y_train = load_images_from_folder('/content/Folio Leaf Dataset/Folio/train')

In [12]:
from skimage.feature import hog

def feature_extraction(img):
    features, hog_img = hog(img, orientations=9, pixels_per_cell=(8, 8), 
                    cells_per_block=(2, 2), visualize=True, multichannel=True)
    return features, hog_img


In [30]:
def get_feature(data):
    features = []
    for image in data:
        f, hog_img = feature_extraction(image)
        f=f/255
        features.append(f)
    return features
    

In [31]:
X_features = get_feature(X_train)

In [32]:
X_features = np.asarray(X_features, dtype=np.float32)
y_train = np.asarray(y_train)

In [33]:
X_features = np.reshape(X_features, (X_features.shape[0], -1))

In [34]:
X_features[0].shape

(26244,)

In [60]:
from sklearn.svm import SVC

clf = SVC(C=2)
clf.fit(X_features, y_train)

SVC(C=2, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [36]:
X_valid, y_valid = load_images_from_folder('/content/Folio Leaf Dataset/Folio/val')
X_valid = get_feature(X_valid)
X_valid = np.asarray(X_valid, dtype=np.float32)
y_valid = np.asarray(y_valid)
X_valid = np.reshape(X_valid, (X_valid.shape[0], -1))

In [61]:
ypred = clf.predict(X_valid)

In [62]:
from sklearn.metrics import accuracy_score

yscore = accuracy_score(y_valid, ypred)
yscore

0.559748427672956

In [48]:
from sklearn.cluster import KMeans

kmean = KMeans(n_clusters=32)
kmean.fit(X_features)
print(kmean.labels_)

kpred = kmean.predict(X_valid)
kscore = accuracy_score(y_valid, kpred)
kscore

[ 4 21 10 10 21 10 21  4 28 10 21 10  4 21 21  4  4  3  4  4  4  4  4  4
  4  4  4  4  4  4  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 28 20 28
 17 20 20 20 20 17 20 24 20  7 26  2 16 16 16 16  9 16  0  0 16  0  0 16
 16  8 20 23 24 24 18 18 18 18  2 18 18 28 18 26 28 13  5 16 16 16 16 16
 16  9  5 16  0 16 16 16 23 26  5 23 23 23 20 18 23 23 28 23 23 22 26 11
 11 16  7 16  8  8 11  8  8 11 11 11 14 16 14  7 23  7  7  7  7  2  7  5
 31 20  7  2  2  2 29  2 12 29 29 29 29 29 29 29 12 23 12 12 29 12 30 29
 29 29 30 23 19 19 15 19 12 30  7 30 17 28 28  7  7  7 30  7 30 29  2  7
 13 20 28 28 13  9  9 28  9 28  9  9  9  9 13  9  9  9  0  0 28 10 10  3
  0 28  1  0 28 28  0  0  0  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
 25 25  0 28 25 16  0 13 25  9 25 25 25  5 25 16  0  0  1 13  0 13 16 10
  9 13  0  0  5  4  4  4  5  5  5  5 18  3  5  5  5  4  4 23  8  8 23 20
  8  8  8 22 26 26  2  8  9 26  5  5  5  5  5  5  5  5  5  5  5  5  5  5
  5  2 27 31 27 31 31 16 31 31 27 16 31 27 31 31  2

AttributeError: ignored

{0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 16, 18, 20, 21, 22, 23, 24, 25, 26, 28, 29, 31}


In [ ]:
def predict_one_image(path):
    img = image.load_img(path, target_size=(224,224,3))
    img = image.img_to_array(img)
    img=img/255
    img = np.asarray(img, dtype=np.float32)
    img = img.reshape(1, img.shape[0]*img.shape[1]*img.shape[2])
    return img

In [ ]:
path = input()
clf.predict(predict_one_image(path))

/content/Folio Leaf Dataset/Folio/val/coffee/20150324_154930.jpg


array(['coffee'], dtype='<U17')

In [ ]:
/content/Folio Leaf Dataset/Folio/val/lychee/20150714_110426.jpgdatatrain = load_images_from_folder('/content/Folio Leaf Dataset/Folio/train')

In [ ]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "Plants list.pdf"])
CLASS_NAMES

In [ ]:
# ashantiblood = list(data_dir.glob('ashanti blood/*'))

# for image_path in ashantiblood[:3]:
#     display.display(Image.open(str(image_path)))

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img
# create a new generator
imagegen = ImageDataGenerator(rescale=1./255)
# load train data
train = imagegen.flow_from_directory("/content/Folio Leaf Dataset/Folio/train", class_mode="categorical", shuffle=False, batch_size=128, target_size=(224, 224))
# load val data
val = imagegen.flow_from_directory("/content/Folio Leaf Dataset/Folio/val", class_mode="categorical", shuffle=False, batch_size=128, target_size=(224, 224))

In [ ]:
plt.figure(figsize=(18, 4))
plt.plot(model.history['loss'], label = 'train')
plt.plot(model.history['val_loss'], label = 'valid')
plt.legend()
plt.title('Loss Function')
plt.show()